# Linear optimization under constraints

## 01. Imports

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime as dt 
import pulp
from itertools import groupby
import os 
import sys 


## 02. Parameters

In [21]:
parent_directory = os.path.dirname(os.getcwd())
sys.path.append(parent_directory)  
from utils.utils import load_data, transform_data, merge_data
data = load_data()
df_JAN24, df_clients, df_intervenants, df_dispo = data.values()
# data = transform_data(data)
# data = merge_data(data)

In [33]:
df_intervenants

,ID Intervenant,Latitude,Longitude,Compétences,Permis,Véhicule personnel,Dispo / Indispo
0,838320706,48.738516,1.391971,"AIDE MENAGERE, REPAS, TOILETTE",Oui,Oui,"Indispo 01/01, 30/01, 31/01"
1,609468992,48.640555,1.232581,"TOILETTE, REPAS, VIE SOCIALE, AIDE MENAGERE",Oui,Oui,"Indispo Tous les mercredis + 13/01, 14/01, 27/..."
2,78012267,48.729206,1.371985,"HOMMES TOUTES MAINS, JARDINAGE",Oui,Oui,Dispo le 25/01
3,818696864,48.744702,1.357921,"REPAS, AIDE MENAGERE, ACCOMPAGNEMENTS COURSES,...",Oui,Oui,Indispo tous les samedis et dimanche
4,746414886,48.769455,1.197644,"TOILETTE, REPAS, AIDE MENAGERE, ACCOMPAGNEMENT...",Oui,Oui,Indispo tous les mercredis + le 05/01
5,78005437,48.712418,1.356278,"AIDE MENAGERE, ADMINISTRATION, REPAS, TOILETTE...",Oui,Oui,Indispo tous les samedis et dimanche
6,213237245,48.763226,1.241120,"TOILETTE, REPAS, AIDE MENAGERE",Oui,Oui,"Indispo 01/01, 02/01, 03/01, 13/01, 14/01, 16/..."
7,813991780,48.692715,1.073195,"TOILETTE, REPAS, AIDE MENAGERE, ACCOMPAGNEMENT...",Oui,Oui,"Indispo tous les mercredis, vendredi, samedi, ..."
8,856089133,48.645418,1.523077,"ACCOMPAGNEMENTS COURSES PA, REPAS, TOILETTE, A...",Non,Non,"Indispo 05/01, 09/01, 11/01 + du 18/01 au 31/0..."
9,700168298,48.758099,1.210611,"TOILETTE, REPAS, AIDE MENAGERE, ACCOMPAGNEMENT...",Oui,Oui,"Dispo le 01/01, 30/01 et 31/01"


In [42]:
df = pd.DataFrame()
df['col1'] = [1, 2, 3]
df['col2'] = [1, 2, 3]
df = df.rename(columns={'col1': 'colA', 'col2': 'colB'})
print(df)

   colA  colB
0     1     1
1     2     2
2     3     3


In [47]:
distance_matrix_client_client = pd.read_csv('../data/distance_matrix_client_to_client.csv')
distance_matrix_client_client_2 = distance_matrix_client_client.copy()
distance_matrix_client_client_2['ID Client 1'], distance_matrix_client_client_2['ID Client 2'] = distance_matrix_client_client_2['ID Client 2'], distance_matrix_client_client_2['ID Client 1']
distance_matrix_client_client = pd.concat([distance_matrix_client_client, distance_matrix_client_client_2], ignore_index=True)
distance_matrix_inter_client = pd.read_csv('../data/distance_matrix_inter_to_client.csv')

In [30]:
distance_matrix_client_client

,ID Client 1,ID Client 2,geometry_client_1,geometry_client_2,distance_car,duration_car,distance_bike,duration_bike
0,559475456,559277088,POINT (1.3756893 48.7210523),POINT (1.3756893 48.7210523),1 m,1 min,1 m,1 min
1,559475456,87852633,POINT (1.3756893 48.7210523),POINT (1.4987735 48.691944),13.8 km,18 mins,12.1 km,44 mins
2,559277088,87852633,POINT (1.3756893 48.7210523),POINT (1.4987735 48.691944),13.8 km,18 mins,12.1 km,44 mins
3,559475456,243033408,POINT (1.3756893 48.7210523),POINT (1.332848 48.726393),5.9 km,8 mins,4.2 km,18 mins
4,559277088,243033408,POINT (1.3756893 48.7210523),POINT (1.332848 48.726393),5.9 km,8 mins,4.2 km,18 mins
...,...,...,...,...,...,...,...,...
6898,570590313,621952854,POINT (1.3579211 48.7447023),POINT (1.351704 48.7428046),0.8 km,2 mins,0.6 km,2 mins
6899,167937489,621952854,POINT (1.3475838 48.7428036),POINT (1.351704 48.7428046),0.5 km,2 mins,0.5 km,1 min
6900,474721965,621952854,POINT (1.3706525 48.7406195),POINT (1.351704 48.7428046),2.3 km,7 mins,2.3 km,11 mins
6901,751167409,621952854,POINT (1.3388591 48.7293302),POINT (1.351704 48.7428046),2.4 km,6 mins,3.5 km,12 mins


In [49]:
id_client_1 = distance_matrix_client_client.loc[0]['ID Client 1']
id_client_2 = distance_matrix_client_client.loc[0]['ID Client 2']
distance_matrix_client_client[
    (distance_matrix_client_client['ID Client 2']==id_client_1)
    &(distance_matrix_client_client['ID Client 1']==id_client_2)
]

,ID Client 1,ID Client 2,geometry_client_1,geometry_client_2,distance_car,duration_car,distance_bike,duration_bike
6903,559277088,559475456,POINT (1.3756893 48.7210523),POINT (1.3756893 48.7210523),1 m,1 min,1 m,1 min


In [32]:
distance_matrix_inter_client

,ID Intervenant,ID Client,geometry_intervenant,Permis,geometry_client,distance,duration
0,162858075,559475456,POINT (1.3746477 48.7222691),1.0,POINT (1.3756893 48.7210523),375,83
1,78007018,559475456,POINT (1.351054 48.689927),1.0,POINT (1.3756893 48.7210523),5454,497
2,856089133,559475456,POINT (1.5230773 48.6454179),0.0,POINT (1.3756893 48.7210523),17631,3520
3,842621873,559475456,POINT (1.3798422 48.728863),1.0,POINT (1.3756893 48.7210523),1732,250
4,838320706,559475456,POINT (1.3919715 48.7385156),1.0,POINT (1.3756893 48.7210523),3884,326
...,...,...,...,...,...,...,...
2827,818696864,621952854,POINT (1.3579211 48.7447023),1.0,POINT (1.351704 48.7428046),805,149
2828,813991780,621952854,POINT (1.0731951 48.6927148),1.0,POINT (1.351704 48.7428046),23526,1463
2829,739888851,621952854,POINT (1.303906 48.75186),1.0,POINT (1.351704 48.7428046),5135,362
2830,78012267,621952854,POINT (1.3719853 48.7292064),1.0,POINT (1.351704 48.7428046),3234,530


In [ ]:
LOCATIONS = distance_matrix_client_client[['ID Client 1', 'ID Client 2', 'duration_car', 'duration_bike']]
LOCATIONS.rename(mapper={})

In [ ]:
DAY = pd.DataFrame()
DAY['Client'] = df_clients['ID Client']

In [123]:
DAY = pd.DataFrame(
    columns=[
        'Client', 
        'Task', 
        'Start_min', 
        'End_max', 
        'Duration'
    ]
)

new_row = {
    'Client' : '559475456', 
    'Task' : 'REPAS', 
    'Start_min' : '07:00', 
    'End_max' : '21:00', 
    'Duration' : 12 # expressed in nb of 5min timeslots
}
new_row_travel = {
    'Client' : '0', 
}
new_row_waiting = {
    'Client' : '0', 
}
new_row_home = {
    'Client' : '0', 
}


DAY = pd.concat([DAY, pd.DataFrame([new_row_travel]), pd.DataFrame([new_row_waiting]), pd.DataFrame([new_row_home]), pd.DataFrame([new_row])], ignore_index=True)
TASKS_CLIENT = DAY.copy()
TASKS_CLIENT.reset_index(names='task_client_id', inplace=True)
# TASKS_CLIENT['task_client_id'] = TASKS_CLIENT['task_client_id'] + 3
TASKS_CLIENT.set_index('task_client_id', inplace=True)

INTER = pd.DataFrame()
INTER['ID Intervenant'] = [np.random.choice(df_intervenants['ID Intervenant'])]

time_slots = list(range(24*12+2))

/var/folders/0l/m614__wd7_5fy64yf491938r0000gn/T/ipykernel_847/1198596276.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  DAY = pd.concat([DAY, pd.DataFrame([new_row_travel]), pd.DataFrame([new_row_waiting]), pd.DataFrame([new_row_home]), pd.DataFrame([new_row])], ignore_index=True)


In [124]:
TASKS_CLIENT

,Client,Task,Start_min,End_max,Duration
task_client_id,,,,,
0,0,NaN,NaN,NaN,NaN
1,0,NaN,NaN,NaN,NaN
2,0,NaN,NaN,NaN,NaN
3,559475456,REPAS,07:00,21:00,12.0


In [103]:
INTER

,ID Intervenant
0,746414886


In [ ]:
tasks_list = [
    "TOILETTE",
    "REPAS",
    "AIDE MENAGERE",
    "VIE SOCIALE",
    "ACCOMPAGNEMENTS COURSES PA",
    "GARDE D'ENFANTS ",
    "FEMME DE MENAGE",
]

tasks_durations = {
    "TOILETTE": 1,
    "REPAS": 1,
    "AIDE MENAGERE": 2,
    "VIE SOCIALE": 2,
    "ACCOMPAGNEMENTS COURSES PA": 1,
    "GARDE D'ENFANTS ": 2,
    "FEMME DE MENAGE": 2,
}

In [6]:
DAY = pd.DataFrame(
    columns=[
        'Client', 
        'Task', 
        'Start_min', 
        'End_max', 
        'Duration'
    ]
)

TASKS_CLIENT = pd.DataFrame(
    columns=[
        'Id',
        'Client', 
        'Task', 
        'Duration' # derived from DAY dataframe
    ]
)
# set Id as index
# keep 0 for traveling, 1 for waiting, 2 for being at home

INTER = pd.DataFrame(columns=['ID Intervenant'])

time_slots = list(range(24*12+2)) # 5min time slots from 7 to 21 => 14hours + initial and final timeslots (requested to be at home)

In [127]:
TASKS_CLIENT

,Client,Task,Start_min,End_max,Duration
task_client_id,,,,,
0,0,NaN,NaN,NaN,NaN
1,0,NaN,NaN,NaN,NaN
2,0,NaN,NaN,NaN,NaN
3,559475456,REPAS,07:00,21:00,12.0


## 03. Variables definition

In [96]:
def has_car(inter):
    return df_intervenants[df_intervenants['ID Intervenant']==inter]['Véhicule personnel'].values[0]=='Oui'

def get_time_travel(inter, prec_client_task, next_client_task):
    # returns the nb of 5min time slots needed for the travel
    # /!\ not a duration
    if prec_client_task == 2:
        time_travel = distance_matrix_inter_client[
            (distance_matrix_inter_client['ID Intervenant']==inter)
            &(distance_matrix_inter_client['ID Client']==int(TASKS_CLIENT.loc[next_client_task]['Client']))
        ]['duration'].values[0]

    elif next_client_task == 2:
        time_travel = distance_matrix_inter_client[
            (distance_matrix_inter_client['ID Intervenant']==inter)
            &(distance_matrix_inter_client['ID Client']==int(TASKS_CLIENT.loc[prec_client_task]['Client']))
        ]['duration'].values[0]

    else :
        row = distance_matrix_client_client[
            (distance_matrix_client_client['ID Client 2']==int(TASKS_CLIENT.loc[prec_client_task]['Client']))
            &(distance_matrix_client_client['ID Client 1']==int(TASKS_CLIENT.loc[next_client_task]['Client']))
        ]
        time_travel = has_car(inter)*row['duration_car'].values[0] + (1-has_car(inter))*row['duration_bike'].values[0]

    time_travel = time_travel/60
    time_travel = (time_travel//5) + (time_travel%5!=0)*1
    if time_travel==0 : time_travel = 1
    return int(time_travel)


In [104]:
prec_client_task = 2
next_client_task = 3
inter = np.random.choice(INTER['ID Intervenant'])

get_time_travel(inter, prec_client_task, next_client_task)

5

In [130]:
# Define PuLP problem
prob = pulp.LpProblem('PLANNING-OPT', pulp.LpMaximize)

# Variables are : for each intervenant, a list of actions x client (with a 5 min granularity)
variables_dict = {
    id_inter: {
        k: pulp.LpVariable(
            cat="Integer",
            lowBound=TASKS_CLIENT.index.min(),
            upBound=TASKS_CLIENT.index.max(),
            name=f"inter_{id_inter}_{k}",
        )
        for k in time_slots
    }
    for id_inter in INTER['ID Intervenant']
}
for id_inter in variables_dict:
    prob += variables_dict[id_inter][0] == 2 # starts the day at home
    prob += variables_dict[id_inter][time_slots[-1]] == 2 # ends the day at home

is_traveling = {
    id_inter: {
        k: pulp.LpVariable(
            cat='Binary',
            lowBound=0,
            upBound=1,
            name=f"is_traveling_{k}",
        )
        for k in time_slots
    }
    for id_inter in INTER['ID Intervenant']
}
for id_inter in is_traveling:
    for k in time_slots:
        prob += is_traveling[id_inter][k] == 1-(variables_dict[id_inter][k]==0)
    #prob += is_traveling[id_inter] == [int(task_client!=0) for task_client in variables_dict[id_inter].values()]

is_waiting = {
    id_inter: {
        k: pulp.LpVariable(
            cat='Binary',
            lowBound=0,
            upBound=1,
            name=f"is_waiting_{k}",
        )
        for k in time_slots
    }
    for id_inter in INTER['ID Intervenant']
}
for id_inter in is_waiting:
    for k in time_slots:
        prob += is_waiting[id_inter][k] == 1-(variables_dict[id_inter][k]==1)
    # prob += is_waiting[id_inter] == [int(task_client!=1) for task_client in variables_dict[id_inter].values()]

# is_at_same_client = {
#     id_inter: {
#         k: pulp.LpVariable(
#             cat='Binary',
#             lowBound=0,
#             upBound=1,
#             name=f"is_at_same_client_{k}",
#         )
#         for k in time_slots
#     }
#     for id_inter in INTER['ID Intervenant']
# }
# for id_inter in is_at_same_client:
#     for k in time_slots[1:]:
#         prob += is_at_same_client[id_inter][k] == TASKS_CLIENT.loc[variables_dict[id_inter][k]]['Client']==TASKS_CLIENT.loc[variables_dict[id_inter][k-1]]['Client']

is_at_home = {
    id_inter: {
        k: pulp.LpVariable(
            cat='Binary',
            lowBound=0,
            upBound=1,
            name=f"is_waiting_{k}",
        )
        for k in time_slots
    }
    for id_inter in INTER['ID Intervenant']
}
for id_inter in is_at_home:
    for k in time_slots:
        prob += is_at_home[id_inter][k] == 1-(variables_dict[id_inter][k]==2)
    # prob += is_at_home[id_inter] == [int(task_client!=2) for task_client in variables_dict[id_inter].values()]


clients = {
    id_inter: {
        k: pulp.LpVariable(
            cat='Integer',
            lowBound=0,
            upBound=INTER['ID Intervenant'].max(),
            name=f"is_waiting_{k}",
        )
        for k in time_slots
    }
    for id_inter in INTER['ID Intervenant']
}

for id_inter in clients:
    for k in time_slots:
        prob += clients[id_inter][k] == int(TASKS_CLIENT.loc[variables_dict[id_inter][k]]['Client'])




#################
## CONSTRAINTS ##
#################

for inter, variable in variables_dict.items(): # for each client
    
    # if activity chages between 2 timeslots, the intervenant is either traveling, waiting or doing another activity at the same client
    for i in range(len(time_slots)-1):
        if (variable[i+1] != variable[i])&(TASKS_CLIENT.loc[variable[i+1]]['Client']!=TASKS_CLIENT.loc[variable[i+1]]['Client']):
            prob += is_traveling[inter][i+1]+is_waiting[inter][i+1] + is_traveling[inter][i] + is_waiting[inter][i] == 1
        
        # having 0-1-0 is absurd (traveling - waiting - traveling)
        ## if there is a wait, we define by default that it shall be after a travel
        ## therefore anything following a wait shall be either a wait or a task
        if variable[i]==1:
            prob += variable[i+1]!=2 # cannot wait before home
            prob += variable[i+1]>=1
    
    # list of successive similar positions
    grouped_variables = [list(group) for _, group in groupby(list(variable))]
    # unique_client_tasks = [var[0] for var in grouped_variables]
    j=0
    for index, sublist in enumerate(grouped_variables):
        activity = sublist[0]
        j_2 = j+len(sublist)
        
        # successive timeslots doing 1 client_task shall match the task duration
        if activity > 2: # intervention at a client
            exp_duration = int(TASKS_CLIENT.loc[activity]['Duration'])
            for i in range(j, j+exp_duration-1):
                prob += variable[i+1] == variable[i]
            prob += variable[j+exp_duration] != activity # intervenant moves to another activity after due time
        
        # traveling shall match the expected travel duration
        if activity == 0: #traveling
            prec_act = variable[j-1]
            next_act = variable[j_2]
            if next_act==1 : next_act = variable[j_2+len(grouped_variables[index+1])] # case where traveling preceeds a waiting time
            exp_duration = get_time_travel(inter, prec_act, next_act)
            for i in range(j, j+exp_duration-1):
                prob += variable[i+1] == variable[i]
            prob += variable[j+exp_duration]>=1 # after traveling, other activity
        
        j = j_2

KeyError: inter_78005437_0

In [112]:
df = pd.DataFrame()
df['col1'] = [1, 2, 3]
df['col1'].tolist()

[1, 2, 3]

In [ ]:
# Matching timings (other method, a priori non linear)
for inter, variable in variables_dict.items(): # for each client
    grouped_variables = [list(group) for _, group in groupby(list(variable))]
    # successive timeslots doing 1 client_task shall match the task duration
    for sublist in grouped_variables:
        client_task_id = sublist[0]
        duration = TASKS_CLIENT.loc[client_task_id]['Duration']
        prob += len(sublist) == duration // 5
    
    # traveling shall match the expected travel duration
    for i, sublist in enumerate(grouped_variables[1:-1]):
        prec_client_task = grouped_variables[i-1][0]
        next_client_task = grouped_variables[i+1][0]
        if sublist[0]==0: # if the intervenant is traveling
            prob += len(sublist) == get_time_travel(inter, prec_client_task, next_client_task)

In [51]:
dict_test = {
    0 : 1, 
    1 : 1, 
    2 : 1, 
    3 : 2
}

## 04. Optimization

In [ ]:
# OBJECTIVE FUNCTION

'''
we want to limit :
    - The travels of the intervenants => total sum of traveling time
    
we want to maximize :
    - The sum of the tasks done 
'''

prob += pulp.lpSum([
    - is_traveling[id_inter][k] # minimizes the distances traveled
    - is_waiting[id_inter][k] # minimize the waiting time
    - is_at_home[id_inter][k] # maximize the number of interventions performed
    for id_inter in INTER['ID Intervenant'] for k in time_slots]
)

prob.solve()

In [ ]:
def get_duration(client_task_id):
    return TASKS_CLIENT.loc[client_task_id]['Duration']